# Erhebung und Analyse von Sensordaten - Time of flight Kamera


## Step 1: Load data
Get the input data sets inside "Messdaten" as a dictionary with the following structure:
```json
{
    "Input_path_1": np.array,
    "Input_path_2": np.array,
    // ...
    
}
```
Where `np.array` is a three dimensional array containing depth data frames of size 80 * 60 

In [ ]:
import utilities


input_file_paths = utilities.load_input_file_paths()
data_sets = utilities.load_depth_data_sets(input_file_paths)


## Step 2: Visualize the data

Save the loaded data frames as gif files to `output/<input-file-name>/raw.gif` and display them in this notebook.

In [ ]:
from IPython.display import Image, Markdown, display

display(Markdown('### Heatmap animations created for input data:'))

for file_name, frames in data_sets.items():
    try:
        output_path = utilities.create_gif_from_frames(frames, f'output/{file_name}/raw.gif')
    except ValueError as e:
        print(e)
        continue

    display(Image(filename=output_path, width=200), Markdown(f'*{file_name}*'))


## Step 3: Manipulate the data

1. Create a background map for each frame by calculating the mean depth map per sequence. 
2. Use the background map to subtract it from each frame in the sequence
3. Create a foreground mask:  
If the remaining depth value of a pixel is still bigger than a certain threshold, the pixel is considered foreground (value 1)
Otherwise the pixel is part of the background (value 0)

> The foreground masks are saved in a data structure similar to the original frame data

In [ ]:
import filters
import numpy as np

display(Markdown('### Foreground masks created for input data:'))

foreground_masks = {}

for file_name, frames in data_sets.items():
    background = filters.compute_background(frames)

    foreground_mask = [filters.subtract_background_from_frame(frame, background, 1000) for frame in frames]
    foreground_mask = np.stack(foreground_mask)
    
    foreground_masks[file_name] = foreground_mask
    
    try:
        output_path = utilities.create_gif_from_frames(foreground_mask, f'output/{file_name}/foreground_mask.gif', 0, 1)

        display(Image(filename=output_path, width=200))
        display(Markdown(f'*{file_name}*'))
    except ValueError as e:
        print(e)
        continue

## Step 4: Analyze the data

Use cv2 to count shapes in each frame and count the shape as person if it has a certain size

In [ ]:
import vision

display(Markdown(f'*Counting people per sequence..*'))

people_counts = {}

for file_name, foreground_mask in foreground_masks.items():
    people_counts[file_name] = [vision.count_people(mask) for mask in foreground_mask]

    print(f'Max people in frame ({file_name}):', max(people_counts[file_name]))

Label each frame with the amount of people detected.

In [ ]:
# TODO:

for file_name in data_sets:
    utilities.add_text_to_gif(f'output/{file_name}/raw.gif')